In [1]:
from auxiliary import loadData_staticTargetAddrMatch, prepare_data_loaders
from algorithms import RssPosAlgo_NeuralNet_CNNv1
from algorithms import RssPosAlgo_NeuralNet_supervisedTrainingLoop
from algorithms import RssPosAlgo_NearestNeighbour_Interpolation
from algorithms import RssPosAlgo_NearestNeighbour_GetKmeansDb
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

### prepare model, dataloaders and training parameters

In [2]:
datajsonpath = "../experiments/exp004_20241022_sna_kadirburakerdem/data-tshark/data.json"
inp_rss_vals, gt_locations = loadData_staticTargetAddrMatch(datajsonpath, second_hold = 5, shuffle=False, 
                                                            target_addresses=["d8:47:32:eb:6c:38",
"50:c7:bf:19:e6:4d",
"18:28:61:3d:94:7a"], snap250ms=False)

epochs           = 201
batch_size       = 64
train_test_split = 0.8
kernelsize       = 9
window_size      = 50

train_loader, test_loader, xtr, ytr, xts, yts = prepare_data_loaders(inp_rss_vals, gt_locations, 
                                                                     batch_size = batch_size, window_size=window_size,
                                                                     train_test_split = train_test_split,
                                                                     cnn_data=True, cnn_validpaddingkernelsize=kernelsize)

print("Subset sizes | train:", xtr.shape, ", test:",xts.shape)
cnn = RssPosAlgo_NeuralNet_CNNv1(kernel_size=kernelsize, inch=3)
cnn.train()
cnn.cuda()
cnn.device="cuda"

cnn_criterion = nn.MSELoss(reduction='mean')
cnn_optimizer = optim.Adam(cnn.parameters(), lr=1e-3)

Subset sizes | train: torch.Size([1998, 3]) , test: torch.Size([500, 3])


In [3]:
for dat in train_loader:
    print(dat[1].shape)
    break

torch.Size([64, 2, 18])


### train model

In [4]:
sum(p.numel() for p in cnn.parameters() if p.requires_grad)

874

In [5]:
cnn = RssPosAlgo_NeuralNet_supervisedTrainingLoop(train_loader = train_loader, 
                                                  test_loader  = test_loader,
                                                  model        = cnn, 
                                                  criterion    = cnn_criterion, 
                                                  optimizer    = cnn_optimizer, 
                                                  epochs       = epochs,
                                                  testfreq     = 20) # testfreq is in epochs

Epoch [1/201] test loss: 8.086, training loss: -1.000
Epoch [21/201] test loss: 0.939, training loss: 1.075
Epoch [41/201] test loss: 1.528, training loss: 1.064
Epoch [61/201] test loss: 1.130, training loss: 1.042
Epoch [81/201] test loss: 1.008, training loss: 1.030
Epoch [101/201] test loss: 1.119, training loss: 1.036
Epoch [121/201] test loss: 1.075, training loss: 1.043
Epoch [141/201] test loss: 1.362, training loss: 1.020
Epoch [161/201] test loss: 1.141, training loss: 1.027
Epoch [181/201] test loss: 0.887, training loss: 1.005
Epoch [201/201] test loss: 0.956, training loss: 1.044


### save the model

In [6]:
torch.save(cnn.state_dict(), 'savedmodels/dev008_exp004_cnn.pth')

### evaluate model vs interp

In [7]:
cnn = RssPosAlgo_NeuralNet_CNNv1(kernel_size=kernelsize, inch=3)
cnn.load_state_dict(torch.load('savedmodels/dev008_exp004_cnn.pth'));
cnn.eval();

In [8]:
error = 0  # Initialize the error accumulator
for test_inputs, test_labels in test_loader:
    loc_pred_cnn = cnn(test_inputs)
    loc_pred_cnn_np = loc_pred_cnn.detach().numpy()  # Predicted values
    yts_batch_np = test_labels.detach().numpy()  # True values
    diff = yts_batch_np - loc_pred_cnn_np
    error1 = 0
    for i in range(diff.shape[-1]):
        error1 += np.linalg.norm(diff[:,:,i])
    error += error1/diff.shape[-1]

print("Mean error:", error/len(test_loader))

Mean error: 1.445435302613345


In [9]:
db_kmeans = RssPosAlgo_NearestNeighbour_GetKmeansDb(xtr, ytr, num_clusters=3)
meanerror_nene_interp = 0;
for test_idx, x_test_sample in enumerate(xts): 
    loc_pred_nene_interp   = RssPosAlgo_NearestNeighbour_Interpolation(x_test_sample, db_kmeans)
    meanerror_nene_interp += np.linalg.norm((yts[test_idx].numpy() - loc_pred_nene_interp))
print("NeNe+Interp:", np.linalg.norm(meanerror_nene_interp/(test_idx+1)))

NeNe+Interp: 1.636158359643209
